In [0]:

from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

    # source_path = 's3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/'
dest_path = 's3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json_month.v3/date={}'

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review.v1/fact/process_granularity=monthly/", depth_is_1=True)
print path_list[153]
#for path in path_list[153:]:


In [0]:

import pyspark.sql.functions as F

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf


source_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/'
dest_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json.v3/date={}-01/'



s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_load_temp/review_agg_v3/", depth_is_1=True)
print path_list[90]
# def generate_path():
    
# review_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/event_month=2019-01/")
# review_df.registerTempTable("reviews")
# ios_samples_df = spark.sql("SELECT * FROM reviews WHERE market_code = 'apple-store' and user_purchased is not null LIMIT 100 ").cache()
# ios_samples_df.count()
# android_samples_df = spark.sql("SELECT * FROM reviews WHERE market_code = 'google-play' and country_code = 'KR' LIMIT 50").cache()
# android_samples_df.count()
# samples = []
# for idx, df in [
#     ("int-ss-review_v1-apple-store-ios-all-201901", ios_samples_df), 
#     ("int-ss-review_v1-google-play-android-all-201901", android_samples_df)]:
#     print df.columns
#     rows = (
#         df
#         .withColumn("date", F.date_format("time", "yyyy-MM-dd"))
#         .drop("time")
#         .withColumn("chinese_content", F.when((df.content_language == "zh") | (df.content_language == "zh_Hant"), df.content))
#         .withColumn("chinese_reply", F.when((df.reply_language == "zh") | (df.reply_language == "zh_Hant"), df.reply))
#         .withColumn("chinese_title", F.when((df.title_language == "zh") | (df.title_language == "zh_Hant"), df.title))
#         .withColumn("english_content", F.when(df.content_language == "en", df.content))
#         .withColumn("english_reply", F.when(df.reply_language == "en", df.reply))
#         .withColumn("english_title", F.when(df.title_language == "en", df.title))
#         .withColumn("japanese_content", F.when(df.content_language == "ja", df.content))
#         .withColumn("japanese_reply", F.when(df.reply_language == "ja", df.reply))
#         .withColumn("japanese_title", F.when(df.title_language == "ja", df.title))
#         .withColumn("korean_content", F.when(df.content_language == "ko", df.content))
#         .withColumn("korean_reply", F.when(df.reply_language == "ko", df.reply))
#         .withColumn("korean_title", F.when(df.title_language == "ko", df.title))
#         .withColumn("other_language_content", F.when(
#             (df.content_language != "zh") & (df.content_language != "zh_Hant") & (df.content_language != "en") &
#             (df.content_language != "ja") & (df.content_language != "ko"), df.content))
#         .withColumn("other_language_reply", F.when(
#             (df.reply_language != "zh") & (df.reply_language != "zh_Hant") & (df.reply_language != "en") &
#             (df.reply_language != "ja") & (df.reply_language != "ko"), df.reply))
#         .withColumn("other_language_title", F.when(
#             (df.title_language != "zh") & (df.title_language != "zh_Hant") & (df.title_language != "en") &
#             (df.title_language != "ja") & (df.title_language != "ko"), df.title))
#         .collect()
#     )
#     samples.append((idx, rows))
# print samples


In [0]:

import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get
def compare(doc, row):
    # print doc, row
    keys = [u"review_id", u'_identifier', u'app_id', u'chinese_content', u'chinese_reply', u'chinese_title', u'content', u'content_language', u'country_code', u'date', u'device_code', u'english_content', u'english_reply', u'english_title', u'japanese_content', u'japanese_reply', u'japanese_title', u'korean_content', u'korean_reply', u'korean_title', u'language', u'market_code', u'other_language_content', u'other_language_reply', u'other_language_title', u'product_version', u'rating', u'reply', u'reply_date', u'reply_language', u'title', u'title_language', u'user_device', u'user_id', u'user_language', u'user_name', u'user_purchased']
    row_dict =  row.asDict()
    for key in keys:
        # print key
        if key in ("app_id", "_identifier"):
            assert str(doc[key]) == str(row_dict[key]), "{} is diffrent".format(key)
        else:
            assert doc[key] == row_dict[key], "{} is diffrent".format(key)
def es_doc(index, doc_id, routing_id):
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp","C38vEJEuraCw")
    
    url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_doc/{}?routing={}".format(index, doc_id, routing_id)
    resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    doc = json.loads(resp.text)
    source = doc['_source']
    source['review_id'] = doc["_id"]
    return source
for idx, rows in samples:
    for row in rows:
        doc = es_doc(idx, row.review_id, row.app_id)
        compare(doc, row)


In [0]:


#spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_load_temp_v4/event_month=2019-01").filter("review_id='gp:AOqpTOE159oPNXHZnRjuicEGJ6mXPt_GlRDO9ccvT2ulP0aR1nP4VD_BL3mRLka9bzTV6o6qAT2Ld6kTVZgBopA'").show(20,False,vertical=True)

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_load_temp/review_agg_v3/event_month=2019-01").filter("review_id='gp:AOqpTOE159oPNXHZnRjuicEGJ6mXPt_GlRDO9ccvT2ulP0aR1nP4VD_BL3mRLka9bzTV6o6qAT2Ld6kTVZgBopA'").show(20,False,vertical=True)

In [0]:

print spark.read.csv("s3://prod_appannie_uniform_public_app_store_data/review-uniform/HOURLY/2019-01-28/06/gp/",sep="\t").selectExpr("_c0 as id", "_c1 as product_id", "_c2 as product_version", "_c3 as user_id", "_c4 as user_name" , "_c5 as user_language", "_c6 as user_device", "_c7 as user_purchased", "_c8 as rating", "_c9 as country", "_c10 as date","_c11 as language", "_c12 as title", "_c13 as title_t", "_c14 as content", "_c15 as content_t", "_c16 as reply", "_c17 as reply_t", "_c18 as reply_date").filter("id='gp:AOqpTOE159oPNXHZnRjuicEGJ6mXPt_GlRDO9ccvT2ulP0aR1nP4VD_BL3mRLka9bzTV6o6qAT2Ld6kTVZgBopA'").show(1,False,vertical=True)

# [Row(_identifier=220200122023733641, app_id=1195757615, content=u'quando fico sem net, esse jogo fica fechando toda hora. desinstalei.continua na mesma D=', content_language=u'pt', country_code=u'BR', language=u'pt', product_version=u'1.1.1', rating=1, reply=u'Hey! we are working to fix this! thanks! Hope you can give it a second chance in the future!', reply_date=u'2018-12-18T00:00:00.000000Z', reply_id=u'6449885', reply_language=u'en', review_id=u'3691811297', title=u'fechando.', title_language=u'en', user_device=None, user_id=None, user_language=u'English', user_name=None, user_purchased=None, user_review_url=u'https://itunes.apple.com/br/reviews?userProfileId=539705707', process_granularity=u'daily', process_date=datetime.date(2019, 12, 28), process_hour=23, device_code=u'ios-all', market_code=u'apple-store', date=u'2019-01-25', chinese_content=None, chinese_reply=None, chinese_title=None, english_content=None, english_reply=u'Hey! we are working to fix this! thanks! Hope you can give it a second chance in the future!', english_title=u'fechando.', japanese_content=None, japanese_reply=None, japanese_title=None, korean_content=None, korean_reply=None, korean_title=None, other_language_content=u'quando fico sem net, esse jogo fica fechando toda hora. desinstalei.continua na mesma D=', other_language_reply=None, other_language_title=None), Row(_identifier=220200122031050263, app_id=1264572665, content=u'\u79c1\u306f\u5927\u597d\u304d\u3067\u3059\u3002\u65c5\u884c\u306e\u6642\u3001\u3068\u3066\u3082\u3079\u3093\u308a\u3067\u3059\u3002\u2764\ufe0f', content_language=u'ja', country_code=u'VN', language=u'ja', product_version=u'2.6.8', rating=5, reply=u'Thank you for your support!!!', reply_date=u'2019-03-01T00:00:00.000000Z', reply_id=u'7518711', reply_language=u'en', review_id=u'3625761635', title=u'\u3059\u3054\u3044', title_language=u'ja', user_device=None, user_id=None, user_language=u'English', user_name=None, user_purchased=None, user_review_url=u'https://itunes.apple.com/vn/reviews?l=vi&userProfileId=575777468', process_granularity=u'daily', process_date=datetime.date(2019, 12, 29), process_hour=23, device_code=u'ios-all', market_code=u'apple-store', date=u'2019-01-08', chinese_content=None, chinese_reply=None, chinese_title=None, english_content=None, english_reply=u'Thank you for your support!!!', english_title=None, japanese_content=u'\u79c1\u306f\u5927\u597d\u304d\u3067\u3059\u3002\u65c5\u884c\u306e\u6642\u3001\u3068\u3066\u3082\u3079\u3093\u308a\u3067\u3059\u3002\u2764\ufe0f', japanese_reply=None, japanese_title=u'\u3059\u3054\u3044', korean_content=None, korean_reply=None, korean_title=None, other_language_content=None, other_language_reply=None, other_language_title=None),

In [0]:
%%sh

aws s3 ls s3://prod_appannie_uniform_public_app_store_data/review-uniform/HOURLY/2019-01-28/06/gp/ | head -3


In [0]:
%%sh
